В данном ноутбуке сделаны оценки размеров и характеристик <b>public</b> датасетов. Понятно, что эти характеристики не обязаны совпадать с характеристиками датасетов из набора <b>private</b>, по которым будут подведены окончательные итоги. Но пространство харакетристик <b>public</b> датасетов несколько больше, чем у <b>check</b> датасетов, так что такие оценки могут помочь в подготовке корректно отрабатывающих решений. 

Оценки получены с помощью задержек времени: 

- берётся минимальный исполняемый без ошибок код (чтение файла и запись нулевых предсказаний) и засекается время
- добавляется задержка времени за каждую строку или тот или иной тип столбца

In [1]:
import pandas as pd
import numpy as np

In [2]:
train_limits = [300,300,300,300,300,600,1800,1800]
test_limits = [300,300,300,300,300,300,300,300]

In [3]:
train_times_base = [2.457924, 3.565435, 3.541296, 3.451769, 3.557235, 6.523217, 44.531233, 56.652558]
test_times_base = [2.637015, 3.609926, 4.640570, 3.664909, 3.546279, 4.635332, 14.664047, 56.531765]

In [4]:
# общее количество столбцов
train_times_cols = [7.402195, 13.458852, 7.605927, 6.444866, 7.541857, 20.578517, 7*60+12.538013, 60+37.389323]
test_times_cols = [6.672376, 13.642278, 8.577456, 6.644376, 7.653464, 18.592532, 0, 60+35.580226]

In [5]:
train_times_rows = [3.243647, 6.517584, 23.524735, 7.365443, 16.489477, 18.486783, 50.507639, 120+35.529968]
test_times_rows = [3.630711, 5.616317, 24.647365, 8.657998, 7.642431, 9.602436, 15.699040, 120+36.701354]

In [6]:
train_times_strs = [3.321004, 3.517424, 3.511291, 3.326997, 3.614427, 7.436826, 48.422907, 65.445962]
test_times_dates = [3.645378, 3.622774, 4.573447, 3.610938, 4.426323, 5.580422, 15.604530, 62.564044]

In [7]:
df = pd.DataFrame({'train_time_base':train_times_base, 'test_time_base':test_times_base, 
                   'train_time_rows':train_times_rows, 'test_time_rows':test_times_rows,
                   'train_time_cols':train_times_cols, 'test_time_cols':test_times_cols, 
                   'train_time_strs':train_times_strs, 'test_time_dates':test_times_dates,
                   'train_limit':train_limits, 'test_limit':test_limits}, 
                   index = ['public_'+str(i) for i in range(1,9)])

In [8]:
df['train_lenght'] = ((df['train_time_rows'] - df['train_time_base']) / 0.0001).astype(int)
df['test_lenght'] = ((df['test_time_rows'] - df['test_time_base']) / 0.0001).astype(int)

In [9]:
df['train_cols'] = ((df['train_time_cols'] - df['train_time_base']) / 0.1).astype(int)
df['test_cols'] = ((df['test_time_cols'] - df['test_time_base']) / 0.1).astype(int)

In [10]:
df['str_cols'] = ((df['train_time_strs'] - df['train_time_base']) / 0.1).astype(int)
df['date_cols'] = ((df['test_time_dates'] - df['test_time_base']) / 0.1).astype(int)

In [11]:
#исправление ошибок
df.loc['public_7','test_cols'] = df.loc['public_7','train_cols'] # вылетело по таймлимиту
df.loc['public_4','str_cols'] = 0 #получилось -1

In [12]:
df['numerical_cols'] = ((df['train_cols'] - 1 + df['test_cols']) / 2 - df['str_cols'] - df['date_cols']).astype(int)

In [13]:
df[['train_lenght', 'test_lenght', 'train_cols', 'test_cols', 'str_cols', 'date_cols', 'numerical_cols', 'train_limit', 'test_limit']]

,train_lenght,test_lenght,train_cols,test_cols,str_cols,date_cols,numerical_cols,train_limit,test_limit
public_1,7857,9936,49,40,8,10,26,300,300
public_2,29521,20063,98,100,0,0,98,300,300
public_3,199834,200067,40,39,0,0,39,300,300
public_4,39136,49930,29,29,0,0,28,300,300
public_5,129322,40961,39,41,0,8,31,300,300
public_6,119635,49671,140,139,9,9,121,600,300
public_7,59764,10349,3880,3880,38,9,3832,1800,300
public_8,988774,1001695,407,390,87,60,251,1800,300


Аналогичные характеристики <b>check</b> датасетов.

In [14]:
folders = ['check_'+str(i)+'_r' for i in range(1,4)]+['check_'+str(i)+'_c' for i in range(4,9)]

In [15]:
train_lenghts = []
test_lenghts = []
train_cols = []
test_cols = []
str_cols = []
date_cols = []
numerical_cols = []
train_limits = [300, 300, 300, 300, 300, '', '', '']
test_limits = [300, 300, 300, 300, 300, '', '', '']

for folder in folders:
    print(folder, end=' ')
    train = pd.read_csv('../data/'+folder+'/train.csv', low_memory=False)
    train_lenghts.append(train.shape[0])
    train_cols.append(train.shape[1])
    str_cols.append(len([c for c in train.columns.tolist() if c.startswith('string')]))
    date_cols.append(len([c for c in train.columns.tolist() if c.startswith('datetime')]))
    numerical_cols.append(len([c for c in train.columns.tolist() if c.startswith('number')]))
    
    test = pd.read_csv('../data/'+folder+'/test.csv', low_memory=False)
    test_lenghts.append(test.shape[0])
    test_cols.append(test.shape[1])
    del train, test

check_1_r check_2_r check_3_r check_4_c check_5_c check_6_c check_7_c check_8_c 

In [16]:
check = pd.DataFrame({'train_lenght':train_lenghts, 'test_lenght':test_lenghts, 'train_cols':train_cols, 'test_cols':test_cols,
                      'str_cols':str_cols, 'date_cols':date_cols, 'numerical_cols':numerical_cols, 
                      'train_limit':train_limits, 'test_limit':test_limits}, index = folders)

In [17]:
check[['train_lenght', 'test_lenght', 'train_cols', 'test_cols', 'str_cols', 'date_cols', 'numerical_cols', 'train_limit', 'test_limit']]

,train_lenght,test_lenght,train_cols,test_cols,str_cols,date_cols,numerical_cols,train_limit,test_limit
check_1_r,365,172,42,41,0,1,39,300,300
check_2_r,13958,5976,11,10,2,0,6,300,300
check_3_r,250000,146400,43,42,0,1,39,300,300
check_4_c,114130,45385,143,142,0,3,138,300,300
check_5_c,467485,169638,17,16,0,1,14,300,300
check_6_c,108814,72545,114,113,0,0,112,,
check_7_c,92091,87062,774,773,4,2,765,,
check_8_c,143525,61512,878,877,32,91,753,,


Максимальный размер датасета <b>public</b> больше, чем датасета <b>check</b>:
- по количеству строк в 2 раза (~1 млн строк vs ~0.5 млн строк)
- по количеству столбцов в 4 раза (~3880 столбцов vs 878 столбцов)    